# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [3]:
import torch as th

In [4]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [5]:
y = x + x

In [6]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [7]:
import syft as sy

In [8]:
hook = sy.TorchHook(th)

In [9]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [10]:
bob = sy.VirtualWorker(hook, id="bob")

In [11]:
bob._objects

{}

In [12]:
x = th.tensor([1,2,3,4,5])

In [13]:
x = x.send(bob)

In [14]:
bob._objects

{94684914890: tensor([1, 2, 3, 4, 5])}

In [15]:
x.location

<VirtualWorker id:bob #tensors:1>

In [16]:
x.id_at_location

94684914890

In [17]:
x.id

22696946150

In [18]:
x.owner

<VirtualWorker id:me #tensors:0>

In [19]:
hook.local_worker

<VirtualWorker id:me #tensors:0>

In [20]:
x

(Wrapper)>[PointerTensor | me:22696946150 -> bob:94684914890]

In [21]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [22]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [23]:
# try this project here!

In [24]:
alice = sy.VirtualWorker(hook, id="alice")

In [25]:
alice._objects

{}

In [26]:
x=x.send(bob,alice)

In [27]:
print("bob",bob._objects)
print("alice", alice._objects)

bob {32130034886: tensor([1, 2, 3, 4, 5])}
alice {10368436593: tensor([1, 2, 3, 4, 5])}


In [28]:
x.location

In [29]:
print(x.id_at_location)

None


In [30]:
x = x.get()

In [31]:
x

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [32]:
print("bob",bob._objects)
print("alice", alice._objects)

bob {}
alice {}


# Lesson: Introducing Remote Arithmetic

In [33]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [34]:
x

(Wrapper)>[PointerTensor | me:28926424852 -> bob:16364561288]

In [35]:
y

(Wrapper)>[PointerTensor | me:98026460430 -> bob:67505886145]

In [36]:
z = x + y

In [37]:
z

(Wrapper)>[PointerTensor | me:42091606580 -> bob:90447455507]

In [38]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [39]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:93394511553 -> bob:9023743446]

In [40]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [41]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [42]:
z = (x + y).sum()

In [43]:
z.backward()

(Wrapper)>[PointerTensor | me:2874545689 -> bob:63623572265]

In [44]:
x = x.get()

In [45]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [46]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [47]:
# try this project here!

In [48]:
input_tensor = th.tensor([[0,0],[0,1],[1,0],[1.,1]], requires_grad=True).send(bob)
target = th.tensor([[0],[0],[1],[1.]],requires_grad=True).send(bob)

In [49]:
weights = th.tensor([[0.],[0.]],requires_grad=True).send(bob)

In [50]:
predictions = th.mm(input_tensor, weights)

In [51]:
predictions

(Wrapper)>[PointerTensor | me:72774037872 -> bob:92964610909]

In [52]:
loss = ((predictions - target)**2).sum()
loss.backward()
weights.data.sub_(weights.grad * 0.1)
weights.grad *= 0 

In [53]:
loss

(Wrapper)>[PointerTensor | me:41217338933 -> bob:28812655604]

In [54]:
print(loss.get().data)

tensor(2.)


# Lesson: Garbage Collection and Common Errors


In [61]:
bob = bob.clear_objects()

In [62]:
bob._objects

{}

In [63]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [64]:
bob._objects

{40326831986: tensor([1, 2, 3, 4, 5])}

In [65]:
del x

In [66]:
bob._objects

{}

In [67]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [68]:
bob._objects

{81472341576: tensor([1, 2, 3, 4, 5])}

In [69]:
x = "asdf"

In [70]:
bob._objects

{}

In [71]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [72]:
x

(Wrapper)>[PointerTensor | me:55848926247 -> bob:33552626914]

In [73]:
bob._objects

{33552626914: tensor([1, 2, 3, 4, 5])}

In [74]:
x = "asdf"

In [75]:
bob._objects

{33552626914: tensor([1, 2, 3, 4, 5])}

In [76]:
del x

In [77]:
bob._objects

{33552626914: tensor([1, 2, 3, 4, 5])}

In [78]:
bob = bob.clear_objects()
bob._objects

{}

In [79]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [80]:
bob._objects

{87148609262: tensor([1, 2, 3, 4, 5])}

In [81]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [82]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:73980600692 -> bob:45036776775]
Tensor B: tensor([1, 1, 1, 1, 1])

In [83]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [84]:
z = x + y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:bob #tensors:2> while the other is on <VirtualWorker id:alice #tensors:1>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [85]:
from torch import nn, optim

In [86]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [87]:
# A Toy Model
model = nn.Linear(2,1)

In [88]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [89]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(1.8628)
tensor(0.3520)
tensor(0.1622)
tensor(0.0991)
tensor(0.0632)
tensor(0.0405)
tensor(0.0260)
tensor(0.0167)
tensor(0.0107)
tensor(0.0069)
tensor(0.0044)
tensor(0.0028)
tensor(0.0018)
tensor(0.0012)
tensor(0.0008)
tensor(0.0005)
tensor(0.0003)
tensor(0.0002)
tensor(0.0001)
tensor(8.7532e-05)


In [90]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [91]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [92]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [93]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [94]:
train()

tensor(0.3894, requires_grad=True)
tensor(0.5806, requires_grad=True)
tensor(0.2751, requires_grad=True)
tensor(0.3300, requires_grad=True)
tensor(0.1621, requires_grad=True)
tensor(0.1899, requires_grad=True)
tensor(0.0944, requires_grad=True)
tensor(0.1093, requires_grad=True)
tensor(0.0550, requires_grad=True)
tensor(0.0629, requires_grad=True)
tensor(0.0321, requires_grad=True)
tensor(0.0362, requires_grad=True)
tensor(0.0187, requires_grad=True)
tensor(0.0208, requires_grad=True)
tensor(0.0110, requires_grad=True)
tensor(0.0120, requires_grad=True)
tensor(0.0064, requires_grad=True)
tensor(0.0069, requires_grad=True)
tensor(0.0038, requires_grad=True)
tensor(0.0040, requires_grad=True)
tensor(0.0022, requires_grad=True)
tensor(0.0023, requires_grad=True)
tensor(0.0013, requires_grad=True)
tensor(0.0013, requires_grad=True)
tensor(0.0008, requires_grad=True)
tensor(0.0008, requires_grad=True)
tensor(0.0005, requires_grad=True)
tensor(0.0004, requires_grad=True)
tensor(0.0003, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [95]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #tensors:0>

In [96]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [97]:
x = x.send(alice)

In [98]:
bob._objects

{2916808538: tensor([1, 2, 3, 4, 5])}

In [99]:
alice._objects

{31043832046: (Wrapper)>[PointerTensor | alice:31043832046 -> bob:2916808538]}

In [100]:
y = x + x

In [101]:
y

(Wrapper)>[PointerTensor | me:57199056246 -> alice:93495046034]

In [102]:
bob._objects

{2916808538: tensor([1, 2, 3, 4, 5]),
 21015458228: tensor([ 2,  4,  6,  8, 10])}

In [103]:
alice._objects

{31043832046: (Wrapper)>[PointerTensor | alice:31043832046 -> bob:2916808538],
 93495046034: (Wrapper)>[PointerTensor | alice:93495046034 -> bob:21015458228]}

In [104]:
jon = sy.VirtualWorker(hook, id="jon")

In [105]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [106]:
bob._objects

{63942177547: tensor([1, 2, 3, 4, 5])}

In [107]:
alice._objects

{859576301: (Wrapper)>[PointerTensor | alice:859576301 -> bob:63942177547]}

In [108]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:859576301 -> bob:63942177547]

In [109]:
bob._objects

{63942177547: tensor([1, 2, 3, 4, 5])}

In [110]:
alice._objects

{}

In [111]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [112]:
bob._objects

{}

In [113]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [114]:
bob._objects

{39749531848: tensor([1, 2, 3, 4, 5])}

In [115]:
alice._objects

{82084572349: (Wrapper)>[PointerTensor | alice:82084572349 -> bob:39749531848]}

In [116]:
del x

In [117]:
bob._objects

{}

In [118]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [119]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #tensors:0>

In [120]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [121]:
bob._objects

{78503138832: tensor([1, 2, 3, 4, 5])}

In [122]:
alice._objects

{}

In [123]:
x.move(alice)

(Wrapper)>[PointerTensor | me:88853453706 -> alice:88853453706]

In [124]:
bob._objects

{}

In [125]:
alice._objects

{88853453706: tensor([1, 2, 3, 4, 5])}

In [126]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [127]:
bob._objects

{274241313: tensor([1, 2, 3, 4, 5])}

In [128]:
alice._objects

{88853453706: tensor([1, 2, 3, 4, 5]),
 56597433706: (Wrapper)>[PointerTensor | alice:56597433706 -> bob:274241313]}

In [129]:
x.remote_get()

(Wrapper)>[PointerTensor | me:11146966565 -> alice:56597433706]

In [130]:
bob._objects

{}

In [131]:
alice._objects

{88853453706: tensor([1, 2, 3, 4, 5]), 56597433706: tensor([1, 2, 3, 4, 5])}

In [132]:
x.move(bob)

(Wrapper)>[PointerTensor | me:11146966565 -> bob:11146966565]

In [133]:
x

(Wrapper)>[PointerTensor | me:11146966565 -> bob:11146966565]

In [134]:
bob._objects

{11146966565: tensor([1, 2, 3, 4, 5])}

In [135]:
alice._objects

{88853453706: tensor([1, 2, 3, 4, 5])}